In [1]:
import cdms2 as cdms
import MV2 as MV
import cdtime,cdutil,genutil
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import string
import glob
import scipy.stats as stats
# Local solution
# If running remotely, uncomment the following code:
# %%bash
# git clone https://github.com/katemarvel/CMIP5_tools
# import CMIP5_tools as cmip5
import sys,os
#sys.path.append("/Users/kmarvel/Google Drive/python-utils")
sys.path.append("../python-utils")
import CMIP5_tools as cmip5
import DA_tools
import Plotting

from eofs.cdms import Eof
from eofs.multivariate.cdms import MultivariateEof
%matplotlib inline

import requests
import pandas as pd

### Set classic Netcdf (ver 3)
cdms.setNetcdfShuffleFlag(0)
cdms.setNetcdfDeflateFlag(0)
cdms.setNetcdfDeflateLevelFlag(0)

#external_drive='/Volumes/CMIP6/'
external_drive="/home/kdm2144/"

In [2]:
ingrid_cmip6 = pd.read_csv("mary_cmip6.csv")

In [3]:
ingrid_cmip6.columns

Index(['activity_id', 'experiment_id', 'file_basename', 'file_dirname',
       'file_fullpath', 'grid_label', 'institution_id', 'member_id', 'mip_era',
       'source_id', 'table_id', 'time_range', 'variable_id', 'tracking_id',
       'version'],
      dtype='object')

In [4]:
def make_dataframe():
    allvariables=['tas', 'mrro', 'mrros', 'mrso', "mrsos",'pr', 'evspsbl', 'prsn']
    experiments=["piControl","historical","ssp126","ssp585","ssp370","ssp245"]
    d={}
    d["filename"]=[]
    d["model"]=[]
    d["variable"]=[]
    d["rip"]=[]
    d["experiment"]=[]
    #for variable in allvariables:
    for variable in allvariables:

        direc="/home/kdm2144/DROUGHT/DOWNLOADED_RAW/"+variable+"/"
        models=sorted(os.listdir(direc))

        for model in models:

            #experiments=np.unique([x.split(".")[1] for x in os.listdir(direc+model+"/")])
            for experiment in experiments:
                filenames=glob.glob(direc+model+"/*."+experiment+".*")
                for fname in filenames:
                    d["filename"]+=[fname]
                    d["model"]+=[model]
                    d["variable"]+=[variable]
                    d["rip"]+=[fname.split(".")[3]]
                    d["experiment"]+=[experiment]
    df=pd.DataFrame(d)
    return df


In [5]:
df=make_dataframe()

In [6]:
def whats_available(model,df):
    MD={}
    allvariables=['tas', 'mrro', 'mrros', 'mrso', 'pr', 'evspsbl', 'prsn',"mrsos"]
    experiments=["piControl","historical","ssp126","ssp585","ssp370","ssp245"]
    for variable in allvariables:
        MD[variable]={}
        for experiment in experiments:
            rips=np.unique(df[(df.model==model) & (df.experiment==experiment) & (df.variable==variable)].rip)
            MD[variable][experiment]=len(rips)
           # for rip in rips:
               # MD[variable][experiment][rip]=len(df[(df.model==model) \
                                                # & (df.experiment==experiment) \
                                            #     & (df.variable==variable) \
                                             #   & df.rip==rip])
    return pd.DataFrame(MD)

In [7]:
def var_available(variable,df):
    MD={}
    #allvariables=['tas', 'mrro', 'mrros', 'mrso', 'pr', 'evspsbl', 'prsn',"mrsos"]
    models = np.array(['ACCESS-CM2', 'ACCESS-ESM1-5', 'CESM2', 'CESM2-WACCM', 'CanESM5',
       'EC-Earth3', 'EC-Earth3-Veg', 'FGOALS-g3', 'HadGEM3-GC31-LL',
       'INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR', 'MIROC6',
       'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0'], dtype='<U15')
    experiments=["piControl","historical","ssp126","ssp585","ssp370","ssp245"]
    for model in models:
        MD[model]={}
        for experiment in experiments:
            rips=np.unique(df[(df.model==model) & (df.experiment==experiment) & (df.variable==variable)].rip)
            MD[model][experiment]=len(rips)
           # for rip in rips:
               # MD[variable][experiment][rip]=len(df[(df.model==model) \
                                                # & (df.experiment==experiment) \
                                            #     & (df.variable==variable) \
                                             #   & df.rip==rip])
    return pd.DataFrame(MD)

In [14]:
whats_available('FGOALS-g3',df)

,tas,mrro,mrros,mrso,pr,evspsbl,prsn,mrsos
historical,3,0,0,0,3,3,3,0
piControl,1,0,0,0,1,1,1,0
ssp126,1,0,1,0,1,1,1,1
ssp245,1,0,1,0,1,1,1,1
ssp370,1,0,0,0,1,1,1,1
ssp585,1,1,0,0,1,1,1,1


In [12]:
direc="/home/kdm2144/DROUGHT/DOWNLOADED_RAW/"+variable+"/"
models=sorted(os.listdir(direc))
models

['ACCESS-CM2',
 'ACCESS-ESM1-5',
 'AWI-CM-1-1-MR',
 'AWI-ESM-1-1-LR',
 'BCC-CSM2-MR',
 'BCC-ESM1',
 'CAMS-CSM1-0',
 'CAS-ESM2-0',
 'CESM2',
 'CESM2-FV2',
 'CESM2-WACCM',
 'CESM2-WACCM-FV2',
 'CIESM',
 'CNRM-CM6-1',
 'CNRM-CM6-1-HR',
 'CNRM-ESM2-1',
 'CanESM5',
 'CanESM5-CanOE',
 'E3SM-1-0',
 'E3SM-1-1',
 'E3SM-1-1-ECA',
 'EC-Earth3',
 'EC-Earth3-LR',
 'EC-Earth3-Veg',
 'EC-Earth3-Veg-LR',
 'FGOALS-f3-L',
 'FGOALS-g3',
 'FIO-ESM-2-0',
 'GFDL-CM4',
 'GFDL-ESM4',
 'GISS-E2-1-G',
 'GISS-E2-1-G-CC',
 'GISS-E2-1-H',
 'GISS-E2-2-G',
 'HadGEM3-GC31-LL',
 'HadGEM3-GC31-MM',
 'IITM-ESM',
 'INM-CM4-8',
 'INM-CM5-0',
 'IPSL-CM6A-LR',
 'KACE-1-0-G',
 'MCM-UA-1-0',
 'MIROC-ES2L',
 'MIROC6',
 'MPI-ESM-1-2-HAM',
 'MPI-ESM1-2-HR',
 'MPI-ESM1-2-LR',
 'MRI-ESM2-0',
 'NESM3',
 'NorCPM1',
 'NorESM1-F',
 'NorESM2-LM',
 'NorESM2-MM',
 'SAM0-UNICON',
 'TaiESM1',
 'UKESM1-0-LL']

In [10]:
variable="pr"
test=var_available(variable,df)
test
#flatex=open("test.tex","w")
#flatex.write(test.T.to_latex())
#flatex.close()

,ACCESS-CM2,ACCESS-ESM1-5,CESM2,CESM2-WACCM,CanESM5,EC-Earth3,EC-Earth3-Veg,FGOALS-g3,HadGEM3-GC31-LL,INM-CM4-8,INM-CM5-0,IPSL-CM6A-LR,MIROC6,MPI-ESM1-2-HR,MPI-ESM1-2-LR,MRI-ESM2-0
historical,2,10,11,3,50,9,5,3,4,1,10,32,50,10,10,6
piControl,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,2
ssp126,1,3,2,1,50,1,4,1,1,1,1,6,3,2,10,1
ssp245,1,3,3,5,50,1,3,1,4,1,1,11,3,2,10,5
ssp370,1,3,2,3,50,1,3,1,0,1,5,11,3,10,9,5
ssp585,1,3,2,5,50,1,4,1,3,1,1,6,3,2,10,2


In [8]:
def check_availability(curr_mod,curr_var,experiment_id):
    ingrid_cmip6 = pd.read_csv("mary_cmip6.csv")
    df1 = ingrid_cmip6[(ingrid_cmip6.source_id==curr_mod)\
                       & (ingrid_cmip6.variable_id==curr_var)\
                       & (ingrid_cmip6.experiment_id == experiment_id)]
    return(df1)
def get_members(curr_mod,curr_var,experiment_id):
    df1=check_availability(curr_mod,curr_var,experiment_id)
    return(np.unique(df1.member_id))

In [9]:
get_members(model,"mrro","ssp585")

NameError: name 'model' is not defined

In [ ]:
def pull_data(curr_mod,curr_var,experiment_id,member_id,overwrite=False):
    # Baseline directory
    base_dir   = 'http://mary.ldeo.columbia.edu:81/CMIP6/.'
    # Write directory
    
    base_write_dir= external_drive+'DROUGHT/DOWNLOADED_RAW/'
    df_proclist = pd.DataFrame(columns=['model','sim','ensemble','variable'])
    ingrid_cmip6 = pd.read_csv("mary_cmip6.csv")


    write_dir = base_write_dir+curr_var+"/"+curr_mod+"/"
    write_stem = curr_var+"."+experiment_id+"."+curr_mod+"."+member_id+".*.nc"
    
    #If the directory doesn't exist already, make it
    os.makedirs(os.path.join(base_write_dir, curr_var, curr_mod),exist_ok=True)
    
    if not overwrite:
        already_exist=glob.glob(write_dir+write_stem)
        if len(already_exist)!=0:
            #print("Already done!")
            return False
    #rips=np.unique(np.array(df1.member_id))

    df1 = ingrid_cmip6[(ingrid_cmip6.source_id==curr_mod)\
                                   & (ingrid_cmip6.variable_id==curr_var) & \
                                   (ingrid_cmip6.experiment_id == experiment_id)&\
                                  (ingrid_cmip6.member_id == member_id)]

    #Construct openDAP link
    nfiles,nidentifiers=df1.shape
    times=np.sort(np.array(df1.time_range))
    i_ens=np.where(df1.time_range==times[0])[0]
    time_range=times[0]

    for time_range in times:
        # Construct Remote OpenDAP Link
        i_ens=np.where(df1.time_range==time_range)[0][0]
        nc_link = base_dir+df1.activity_id.iloc[i_ens]+'/.'+df1.institution_id.iloc[i_ens]+'/.'+curr_mod+'/.'+experiment_id+'/.'+df1.member_id.iloc[i_ens]+'/.'+df1.table_id.iloc[i_ens]+ \
                            '/.'+curr_var+'/.'+df1.grid_label.iloc[i_ens]+'/.'+df1.version.iloc[i_ens]+'/.'+df1.file_basename.iloc[i_ens]+'/.'+curr_var+'/dods'
        request = requests.get(nc_link)
        if request.status_code == 200:
            #Get the data
            f=cdms.open(nc_link)
            data=f(curr_var)
            tax=data.getTime()
            tax.id="time"
            latax=data.getLatitude()
            lonax=data.getLongitude()
            #reshape it to years and months
            ntime=data.shape[0]
            nyears=int(ntime/12)
            rdata=data.reshape((nyears,12)+data.shape[1:])
            for i in range(nyears):
                yeardata=rdata[i]
                #Make the time axis
                tax_trunc=cdms.createAxis(tax[12*i:12*(i+1)])
                tax_trunc.designateTime()
                for key in tax.attributes.keys():
                    setattr(tax_trunc,key,tax.attributes[key])
                yeardata.setAxis(0,tax_trunc)
                yeardata.setAxis(1,latax)
                yeardata.setAxis(2,lonax)
                #get the start year for labeling purposes
                year=str(tax_trunc.asComponentTime()[0].year)
                writename = curr_var+"."+experiment_id+"."+curr_mod+"."+member_id+"."+year.zfill(4)+".nc"
                fw=cdms.open(write_dir+writename,"w")
                fw.write(yeardata)
                fw.close()
            f.close()
        return True


